In [1]:
import numpy as np
import scipy as sp

# Define known parameters
rho1 = 1573.2; mu1 = 65e-3; N1 = 18000; D1 = 51e-3; M1 = 0.85; P1 = 10.4e5
#rho2 = 1000; mu2 = 10e-3
N2 = 8000; D2 = 50e-3
g = 9.807  # Acceleration due to gravity in m/s^2

Q1 = M1/rho1

# Define lower and upper bounds
lb = [1000, 50e-3, 0.1, 2e5]  # Lower bounds
ub = [10000, 200e-3, 1.0, 4e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    rho2, mu2, M2, P2 = x

    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = (M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = (g * P1/(rho1*g) / (N1**2 * D1**2)) - (g * P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3, F4]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1.0, 1.0e-3, 1.0e-3, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([1000, 40e-3, 0.8, 3e5])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
#result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

# Perform global optimization using basinhopping
# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=4000,T=0.5,stepsize=1,disp=True)

/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/optimize/_basinhopping.py:302: OptimizeWarning: Initial guess is not within the specified bounds
  return self.minimizer(self.func, x0, **self.kwargs)
/var/folders/j3/dr9c4mcx23gcdm0215yvb0h00000gn/T/ipykernel_22711/1481183308.py:23: RuntimeWarning: invalid value encountered in sqrt
  F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))


basinhopping step 0: f 3.74785
basinhopping step 1: f 0.16203 trial_f 0.16203 accepted 1  lowest_f 0.16203
found new global minimum on step 1 with function value 0.16203
basinhopping step 2: f 0.16203 trial_f 0.308378 accepted 0  lowest_f 0.16203
basinhopping step 3: f 0.16203 trial_f 5.36421e+11 accepted 0  lowest_f 0.16203
basinhopping step 4: f 0.16203 trial_f 5.3669e+11 accepted 0  lowest_f 0.16203
basinhopping step 5: f 0.158928 trial_f 0.158928 accepted 1  lowest_f 0.158928
found new global minimum on step 5 with function value 0.158928
basinhopping step 6: f 0.158928 trial_f 0.320053 accepted 0  lowest_f 0.158928
basinhopping step 7: f 1.27469e-05 trial_f 1.27469e-05 accepted 1  lowest_f 1.27469e-05
found new global minimum on step 7 with function value 1.27469e-05
basinhopping step 8: f 3.92301e-05 trial_f 3.92301e-05 accepted 1  lowest_f 1.27469e-05
basinhopping step 9: f 0.320249 trial_f 0.320249 accepted 1  lowest_f 1.27469e-05
basinhopping step 10: f 0.320249 trial_f 5.3673

KeyboardInterrupt: 

In [2]:
rho2, mu2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
Q2 = M2/rho2
specific_speed_1 = N1*np.sqrt(M1/rho1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(M2/rho2)/(P2/(rho2*g))**(3/4)
print(f"Ns = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient Prototype = {flow_coeff_1}, Model = {flow_coeff_2}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient Prototype = {head_coeff_1}, Model = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reynolds Number Prototype = {Reysnold_1:.4f}, Model = {Reysnold_2:.4f}, Matching = {100-abs(Reysnold_2-Reysnold_2)/Reysnold_1*100} %")

print(f"\nModel's Shaft Speed is {N2:.2f} RPM\nModel's Impeller Diameter is {D2*1e3:.2f} mm\nModel's Mass Flow Rate is {M2:.2f} Kg/s\nModel's Pressure Across Pump is {P2/1e5:.2f} Bar\nModel's Volumetric Flow Rate is {Q2*1000:.4f} L/s")

Ns = 10.4453, Matching = 58.73094775368929 %
Flow Coefficient Prototype = 0.0002262830139178427, Model = 0.0012364138095461337, Matching = -346.40151204413536 %
Head Coefficient Prototype = 0.0007844477815818548, Model = 0.004947843508550223, Matching = -430.74223991975566 %
Reynolds Number Prototype = 1133139.6554, Model = 1133139.6302, Matching = 100.0 %

Model's Shaft Speed is 3731.72 RPM
Model's Impeller Diameter is 50.03 mm
Model's Mass Flow Rate is 0.67 Kg/s
Model's Pressure Across Pump is 2.00 Bar
Model's Volumetric Flow Rate is 0.5779 L/s
